# 构建检索问答链

在 `C3 搭建数据库` 章节，我们已经介绍了如何根据自己的本地知识文档，搭建一个向量知识库。 在接下来的内容里，我们将使用搭建好的向量数据库，对 query 查询问题进行召回，并将召回结果和 query 结合起来构建 prompt，输入到大模型中进行问答。   

## 1. 加载向量数据库

首先，我们加载在前一章已经构建的向量数据库。注意，此处你需要使用和构建时相同的 Emedding。

In [10]:
import sys
sys.path.append("../A3 搭建知识库") # 将父目录放入系统路径中

# 使用智谱 Embedding API，注意，需要将上一章实现的封装代码下载到本地
from zhipuai_embedding import ZhipuAIEmbeddings
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores.chroma import Chroma  

从环境变量中加载你的 API_KEY

In [11]:
from dotenv import load_dotenv, find_dotenv
import os

_ = load_dotenv(find_dotenv())    # read local .env file
zhipuai_api_key = os.environ['ZHIPUAI_API_KEY']

加载向量数据库，其中包含了 ../../data_base/knowledge_db 下多个文档的 Embedding

In [3]:
!pwd 

/Users/samprasyuan/Desktop/AIGC/faster-rag/notebook/第一部分  LLM开发入门/A4 构建 RAG 应用


In [12]:
# 定义 Embeddings
# embedding = ZhipuAIEmbeddings() 
embedding = OpenAIEmbeddings()  

# 向量数据库持久化路径
persist_directory = '../../../data_base/vector_db/chroma'

# 加载数据库
vectordb = Chroma(
    persist_directory=persist_directory,  # 允许我们将persist_directory目录保存到磁盘上
    embedding_function=embedding
)

/Users/samprasyuan/miniforge3/envs/simple-rag/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [13]:
print(f"向量库中存储的数量：{vectordb._collection.count()}")

向量库中存储的数量：20


我们可以测试一下加载的向量数据库，使用一个问题 query 进行向量检索。如下代码会在向量数据库中根据相似性进行检索，返回前 k 个最相似的文档。

> ⚠️使用相似性搜索前，请确保你已安装了 OpenAI 开源的快速分词工具 tiktoken 包：`pip install tiktoken`

In [14]:
question = "什么是prompt engineering?"
docs = vectordb.similarity_search(question,k=3)
print(f"检索到的内容数：{len(docs)}")     # 耗时 1min30s 

检索到的内容数：3


打印一下检索到的内容

In [15]:
for i, doc in enumerate(docs):
    print(f"检索到的第{i}个内容: \n {doc.page_content}", end="\n-----------------------------------------------------\n") 

检索到的第0个内容: 
 price.  It

price. On the bright side, it arrived a day earlier than expected, so I got to play with it myself before

I gave

giving it to my daughter.

这个示例展示了如何利用语言模型强大的语言处理能力实现自动化的语法纠错。类似的方法可以运用于校对各类文本内容，大幅减轻人工校对的工作量，同时确保文本语法准确。掌握运用语言模型进行语法纠正的技巧，将使我们的写作更加高效和准确。

五、综合样例

语言模型具有强大的组合转换能力，可以通过一个Prompt同时实现多种转换，大幅简化工作流程。

下面是一个示例，展示了如何使用一个Prompt，同时对一段文本进行翻译、拼写纠正、语气调整和格式转换等操作。
-----------------------------------------------------
检索到的第1个内容: 
 通过这个例子，我们可以看到大语言模型可以流畅地处理多个转换要求，实现中文翻译、拼写纠正、语气升级和格式转换等功能。

利用大语言模型强大的组合转换能力，我们可以避免多次调用模型来进行不同转换，极大地简化了工作流程。这种一次性实现多种转换的方法，可以广泛应用于文本处理与转换的场景中。

六、英文版

1.1 翻译为西班牙语

python
prompt = f"""
Translate the following English text to Spanish: \Hi, I would like to order a blender"""
response = get_completion(prompt)
print(response)

1.2 识别语种

python
prompt = f"""
Tell me which language this is:Combien coûte le lampadaire?```
"""
response = get_completion(prompt)
print(response)

```

1.3 多语种翻译
-----------------------------------------------------


## 2. 创建一个 LLM

在这里，我们调用 OpenAI 的 API 创建一个 LLM，当然你也可以使用其他 LLM 的 API 进行创建

In [16]:
import os 
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"] 

In [17]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model_name = "gpt-3.5-turbo", temperature = 0)

llm.invoke("请你自我介绍一下自己！")  

AIMessage(content='你好，我是一个智能助手，专注于为用户提供各种服务和帮助。我可以回答问题、提供信息、解决问题等。希望能够帮助到您！如果您有任何问题或需求，请随时告诉我。谢谢！', response_metadata={'token_usage': {'completion_tokens': 84, 'prompt_tokens': 20, 'total_tokens': 104}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-a314e6cc-7e12-418d-916f-f4b2909a3ed7-0', usage_metadata={'input_tokens': 20, 'output_tokens': 84, 'total_tokens': 104})

## 3. 构建检索问答链

In [18]:
from langchain.prompts import PromptTemplate

template = """使用以下上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答
案。最多使用三句话。尽量使答案简明扼要。总是在回答的最后说“谢谢你的提问！”。
{context}
问题: {question}
"""

QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context","question"],
                                 template=template)


再创建一个基于模板的检索链：

In [19]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=vectordb.as_retriever(),
                                       return_source_documents=True,
                                       chain_type_kwargs={"prompt":QA_CHAIN_PROMPT})  


创建检索 QA 链的方法 RetrievalQA.from_chain_type() 有如下参数：
- llm：指定使用的 LLM
- 指定 chain type : RetrievalQA.from_chain_type(chain_type="map_reduce")，也可以利用load_qa_chain()方法指定chain type。
- 自定义 prompt ：通过在RetrievalQA.from_chain_type()方法中，指定chain_type_kwargs参数，而该参数：chain_type_kwargs = {"prompt": PROMPT}
- 返回源文档：通过RetrievalQA.from_chain_type()方法中指定：return_source_documents=True参数；也可以使用RetrievalQAWithSourceChain()方法，返回源文档的引用（坐标或者叫主键、索引）

## 4.检索问答链效果测试

In [20]:
question_1 = "什么是南瓜书？"
question_2 = "王阳明是谁？" 

### 4.1 基于召回结果和 query 结合起来构建的 prompt 效果

In [22]:
result = qa_chain({"query": question_1})
print("大模型+知识库后回答 question_1 的结果：")
print(result["result"])  

大模型+知识库后回答 question_1 的结果：
对不起，我不知道什么是南瓜书。谢谢你的提问！


In [23]:
result = qa_chain({"query": question_2})
print("大模型+知识库后回答 question_2 的结果：")
print(result["result"])

大模型+知识库后回答 question_2 的结果：
抱歉，我不知道王阳明是谁。谢谢你的提问！


### 4.2 大模型自己回答的效果

In [24]:
prompt_template = """请回答下列问题:
                            {}""".format(question_1)

### 基于大模型的问答
llm.predict(prompt_template)  

/Users/samprasyuan/miniforge3/envs/simple-rag/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


'南瓜书是指一本关于南瓜的书籍，通常是指一本介绍南瓜的种植、养护、烹饪等方面知识的书籍。南瓜书也可以指一本以南瓜为主题的文学作品。'

In [25]:
prompt_template = """请回答下列问题:
                            {}""".format(question_2)

### 基于大模型的问答
llm.predict(prompt_template) 

'王阳明（1472年-1529年），字宪，号阳明，江西上饶人，明代著名的哲学家、军事家、教育家、政治家。他提出了“致良知”、“格物致知”等重要思想，强调人的内心本具良知，只要发挥良知，就能认识道德真理。他的思想对后世影响深远，被誉为“阳明心学”的创始人。'

> ⭐ 通过以上两个问题，我们发现 LLM 对于一些近几年的知识以及非常识性的专业问题，回答的并不是很好。而加上我们的本地知识，就可以帮助 LLM 做出更好的回答。另外，也有助于缓解大模型的“幻觉”问题。

## 5. 添加历史对话的记忆功能

现在我们已经实现了通过上传本地知识文档，然后将他们保存到向量知识库，通过将查询问题与向量知识库的召回结果进行结合输入到 LLM 中，我们就得到了一个相比于直接让 LLM 回答要好得多的结果。在与语言模型交互时，你可能已经注意到一个关键问题 - **它们并不记得你之前的交流内容**。这在我们构建一些应用程序（如聊天机器人）的时候，带来了很大的挑战，使得对话似乎缺乏真正的连续性。这个问题该如何解决呢？


## 1. 记忆（Memory）

在本节中我们将介绍 LangChain 中的储存模块，即如何将先前的对话嵌入到语言模型中的，使其具有连续对话的能力。我们将使用 `ConversationBufferMemory` ，它保存聊天消息历史记录的列表，这些历史记录将在回答问题时与问题一起传递给聊天机器人，从而将它们添加到上下文中。

In [26]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(
    memory_key="chat_history",  # 与 prompt 的输入变量保持一致。
    return_messages=True  # 将以消息列表的形式返回聊天记录，而不是单个字符串
)

关于更多的 Memory 的使用，包括保留指定对话轮数、保存指定 token 数量、保存历史对话的总结摘要等内容，请参考 langchain 的 [Memory](https://python.langchain.com.cn/docs/modules/memory/) 部分的相关文档。

## 2. 对话检索链（ConversationalRetrievalChain）

对话检索链（ConversationalRetrievalChain）在检索 QA 链的基础上，增加了处理对话历史的能力。

它的工作流程是:
1. 将之前的对话与新问题合并生成一个完整的查询语句。
2. 在向量数据库中搜索该查询的相关文档。
3. 获取结果后,存储所有答案到对话记忆区。
4. 用户可在 UI 中查看完整的对话流程。

![](../../figures/Modular_components.png)

这种链式方式将新问题放在之前对话的语境中进行检索，可以处理依赖历史信息的查询。并保留所有信
息在对话记忆中，方便追踪。

接下来让我们可以测试这个对话检索链的效果：

使用上一节中的向量数据库和 LLM ！首先提出一个无历史对话的问题“这门课会学习 Python 吗？”，并查看回答。

In [29]:
from langchain.chains import ConversationalRetrievalChain

retriever=vectordb.as_retriever()

qa = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=retriever,
    memory=memory
)
question = "我可以学习到关于提示工程的知识吗？"
result = qa({"question": question})
print(result['answer'])

这门课需要教这方面的知识，因为在当今全球化的环境下，跨语言交流变得越来越频繁和重要。了解和掌握大语言模型的文本转换能力，特别是文本翻译的技能，可以帮助我们打通语言壁垒，实现更便捷的国际商务合作和交流。同时，掌握这方面的知识也有助于开发智能的通用翻译工具，提高跨语言交流的效率和质量。


然后基于答案进行下一个问题“为什么这门课需要教这方面的知识？”：

In [30]:
question = "为什么这门课需要教这方面的知识？"
result = qa({"question": question})
print(result['answer'])

抱歉，我不知道。


可以看到，LLM 它准确地判断了这方面的知识，指代内容是强化学习的知识，也就
是我们成功地传递给了它历史信息。这种持续学习和关联前后问题的能力，可大大增强问答系统的连续
性和智能水平。